<a href="https://colab.research.google.com/github/locastre/pyCERR/blob/main/autosegmentation_CT_Lung_SMIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
# work dir
#workDir = r'/home/jupyter' #
workDir = r'/content' #for Colab # r'AI_workshop/heart_model' # local
os.chdir(workDir)

In [7]:
!git clone https://github.com/cerr/model_installer.git

fatal: destination path 'model_installer' already exists and is not an empty directory.


In [10]:
os.chdir(os.path.join(workDir,'model_installer'))

# Location of dicom data
dataDownloadDir = os.path.join(workDir,'input')

os.makedirs(dataDownloadDir,exist_ok=True)
!./installer.sh -h

      ___           ___           ___           ___     
     /  /\         /  /\         /  /\         /  /\    
    /  /:/        /  /:/_       /  /::\       /  /::\   
   /  /:/        /  /:/ /\     /  /:/\:\     /  /:/\:\  
  /  /:/  ___   /  /:/ /:/_   /  /:/~/:/    /  /:/~/:/  
 /__/:/  /  /\ /__/:/ /:/ /\ /__/:/ /:/___ /__/:/ /:/___
 \  \:\ /  /:/ \  \:\/:/ /:/ \  \:\/:::::/ \  \:\/:::::/
  \  \:\  /:/   \  \::/ /:/   \  \::/~~~~   \  \::/~~~~ 
   \  \:\/:/     \  \:\/:/     \  \:\        \  \:\     
    \  \::/       \  \::/       \  \:\        \  \:\    
     \__\/         \__\/         \__\/         \__\/    
 
Medical Physics Department, Memorial Sloan Kettering Cancer Center, New York, NY
 
Welcome to the CERR segmentation model installer! For usage information, run with -h flag
 
Usage Information: 
	Flags: 
		-i : Flag to run installer in interactive mode (no argument)
		-m : [1-4] Integer number to select model to install. For list of available options, see below. 
		-d 

In [11]:
modelOpt = '4' #CT_Lung_SMIT model
pythonOpt = 'C' #download and use pre-packaged Conda environment

! source ./installer.sh -m {modelOpt} -d {workDir} -p {pythonOpt}

      ___           ___           ___           ___     
     /  /\         /  /\         /  /\         /  /\    
    /  /:/        /  /:/_       /  /::\       /  /::\   
   /  /:/        /  /:/ /\     /  /:/\:\     /  /:/\:\  
  /  /:/  ___   /  /:/ /:/_   /  /:/~/:/    /  /:/~/:/  
 /__/:/  /  /\ /__/:/ /:/ /\ /__/:/ /:/___ /__/:/ /:/___
 \  \:\ /  /:/ \  \:\/:/ /:/ \  \:\/:::::/ \  \:\/:::::/
  \  \:\  /:/   \  \::/ /:/   \  \::/~~~~   \  \::/~~~~ 
   \  \:\/:/     \  \:\/:/     \  \:\        \  \:\     
    \  \::/       \  \::/       \  \:\        \  \:\    
     \__\/         \__\/         \__\/         \__\/    
 
Medical Physics Department, Memorial Sloan Kettering Cancer Center, New York, NY
 
Welcome to the CERR segmentation model installer! For usage information, run with -h flag
 
git clone https://github.com/cerr/CT_Lung_SMIT.git
Cloning into 'CT_Lung_SMIT'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 

In [12]:
%%capture
!pip install pyxnat
! pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git@testing"

In [13]:
inputDicomPath = os.path.join(workDir,'input')
outputDicomPath = os.path.join(workDir, 'output')
sessionPath = os.path.join(workDir, 'session')

if not os.path.exists(outputDicomPath):
  os.makedirs(outputDicomPath, exist_ok = True)

if not os.path.exists(sessionPath):
  os.makedirs(sessionPath, exist_ok = True)

In [14]:
from pyxnat import Interface
import urllib3, shutil
from glob import glob
urllib3.disable_warnings()

xhost = 'https://pixnat.mskcc.org'
user = 'locastre'
scan_dict = [{'proj':proj,'subj':subj,'exp':exp,'scan_list':scan_list}]

In [15]:
def getXNATData(xhost,user,scandict,downloadDir):
  xnat = Interface(xhost, user, verify=False)
  os.makedirs(downloadDir, exist_ok=True)
  for scan_entry in scandict:
    proj = scan_entry['proj']
    subj = scan_entry['subj']
    exp = scan_entry['exp']
    scan_list = scan_entry['scan_list']
    expdir = os.path.join(downloadDir,exp)
    os.makedirs(expdir, exist_ok = True)
    xexp = xnat.select.project(proj).subject(subj).experiment(exp)
    for scan in scan_list:
      try:
        xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('DICOM').get(downloadDir,extract=True)
      except:
        xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('secondary').get(downloadDir,extract=True)
    for dcmfolder in ['DICOM','secondary']:
      dcmlist = glob(os.path.join(downloadDir,dcmfolder,'*.dcm'))
      print(dcmlist)
      for dcm in dcmlist:
        shutil.move(dcm, expdir)
  for dcmfolder in ['DICOM','secondary']:
    if os.path.exists(os.path.join(downloadDir,dcmfolder)):
      os.rmdir(os.path.join(downloadDir,dcmfolder))
    if os.path.exists(os.path.join(downloadDir,dcmfolder + '.zip')):
      os.remove(os.path.join(downloadDir,dcmfolder + '.zip'))
  xnat.disconnect()

In [16]:
getXNATData(xhost,user,scandict,inputDicomPath)

··········
['/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-88-ehcqd8.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-54-b7430f.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-36-1sed2tf.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-82-17cuxyi.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-57-474cq4.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-43-iwcq5k.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-6-15leg7.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-93-1878ffd.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.5.2.1.6329.6468.109218621610560682044382179111-1-66-1x4clap.dcm', '/content/input/DICOM/1.3.6.1.4.1.14519.

In [18]:
from glob import glob
dcmdirlist = glob(os.path.join(inputDicomPath,'*'))

['/content/input/SPI_XNAT_E00347', '/content/input/SPI_XNAT_E00348']


In [19]:
import subprocess

In [22]:
wrapperInstallDir = os.path.join(workDir,'CT_Lung_SMIT')
condaEnvDir = os.path.join(wrapperInstallDir,'cpack')
condaEnvActivateScript = os.path.join(condaEnvDir, 'bin', 'activate')
wrapperPath = os.path.join(wrapperInstallDir,'bash_run_SMIT_Segmentation.sh')
load_weight_name = os.path.join(wrapperInstallDir,'trained_weights','model.pt')
scanNum = 0
structName = 'GTV'

In [ ]:

for dcmdir in dcmdirlist:
  subj = os.path.basename(dcmdir)
  sessiondir = os.path.join(sessionPath,subj)
  os.makedirs(sessiondir,exist_ok=True)

  scan_niifile = os.path.join(sessiondir,'CT.nii')
  planC = pc.load_dcm_dir(dcmdir)
  planC.scan[scanNum].save_nii(scan_niifile, planC)

  subprocess.run(f"source " + condaEnvActivateScript + " && source " + wrapperPath + " " + sessiondir + " " + sessiondir + " " + load_weight_name, shell=True, executable="/bin/bash")


In [25]:
dcmdir = dcmdirlist[0]
subj = os.path.basename(dcmdir)
sessiondir = os.path.join(sessionPath,subj)
os.makedirs(sessiondir,exist_ok=True)

scan_niifile = os.path.join(sessiondir,'CT.nii')
planC = pc.load_dcm_dir(dcmdir)
planC.scan[scanNum].save_nii(scan_niifile)

subprocess.run(f"source " + condaEnvActivateScript + " && source " + wrapperPath + " " + sessiondir + " " + sessiondir + " " + load_weight_name, shell=True, executable="/bin/bash")


('0617-292370', '0617-292370', '1.3.6.1.4.1.14519.5.2.1.6329.6468.180596963996965825122162035932', '1.3.6.1.4.1.14519.5.2.1.6329.6468.314188004903460989626820333570', 'CT', 'CT', 'CT', 'CT', 'CT', 'CT', 'CT')
('0617-292370', '0617-292370', '1.3.6.1.4.1.14519.5.2.1.6329.6468.180596963996965825122162035932', '1.3.6.1.4.1.14519.5.2.1.6329.6468.880278915833505391469023528647', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT')


CompletedProcess(args='source /content/CT_Lung_SMIT/cpack/bin/activate && source /content/CT_Lung_SMIT/bash_run_SMIT_Segmentation.sh /content/session/SPI_XNAT_E00347 /content/session/SPI_XNAT_E00347 /content/CT_Lung_SMIT/trained_weights/model.pt', returncode=1)

In [ ]:
#Segmentation stored in NIfTI format can be imported to planC as follows:

niiSeg = r'path/to/niiSeg.nii.gz'
planC = pc.load_nii_scan(niiSeg)
assocScanNum = 0 # index of scan to associate segmentation
structName = 'tumor'
planC = pc.load_nii_structure(niiSeg , assocScanNum, planC, labels_dict={1: structName})